In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import pickle
import pandas as pd

In [ ]:
training_file = 'train.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)

X_train, y_train = train['features'], train['labels']

In [ ]:
n_train = len(X_train)
image_shape = X_train[0].shape

print("Number of training examples =", n_train)
print("Image data shape =", image_shape)

In [ ]:
signnames = pd.read_csv('signnames.csv')
signnames.set_index('ClassId',inplace=True)

def get_name_from_label(label):
    return signnames.loc[label].SignName

In [ ]:
X_train_gry = np.sum(X_train/3, axis=3, keepdims=True)

X_train = X_train_gry

print(X_train.shape)

In [ ]:
X = tf.placeholder(tf.float32, [None, 32, 32, 1])
Y = tf.placeholder(tf.int32, [None])
one_hot_Y = tf.one_hot(Y,43)
is_training = tf.placeholder(tf.bool)

conv1=tf.layers.conv2d(X, 6, [5, 5], activation=tf.nn.relu)
conv1=tf.layers.max_pooling2d(conv1, [2, 2], [2, 2])
conv1 = tf.layers.dropout(conv1, 0.7, is_training)

conv2=tf.layers.conv2d(conv1, 16, [5, 5], activation=tf.nn.relu)
conv2=tf.layers.max_pooling2d(conv2, [2, 2], [2, 2])
conv2 = tf.layers.dropout(conv2, 0.7, is_training)

fc1 = tf.contrib.layers.flatten(conv2)
fc1 = tf.layers.dense(fc1, 400, activation=tf.nn.relu)
fc1 = tf.layers.dropout(fc1, 0.5, is_training)

fc2 = tf.layers.dense(fc1, 800, activation=tf.nn.relu)
fc2 = tf.layers.dropout(fc2, 0.5, is_training)

logits = tf.layers.dense(fc2, 43, activation=None)

In [ ]:
def train():
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = one_hot_Y, logits = logits))
    optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
    sess.run(tf.global_variables_initializer())
  
    batch_size = 128
    epoch = 20
  
    for epoch in range(epoch):
        
        total_cost = 0
        for offset in range(0, len(X_train), batch_size):
            end = offset + batch_size
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            _, cost_val = sess.run([optimizer, cost],feed_dict={X: batch_x, Y: batch_y, is_training: True})
            total_cost += cost_val
        print('Epoch:', '%04d' % (epoch + 1),'Avg. cost =', '{:.4f}'.format(total_cost / (len(X_train)/batch_size)))
    print('최적화 완료!')

In [ ]:
def save_variables():

    train()
    saver = tf.train.Saver()
    saver.save(sess, 'weights/test.ckpt')

In [ ]:
def restore_variables():
    
    saver = tf.train.Saver()
    saver.restore(sess, 'weights/test.ckpt')
    
    index = random.randint(0, len(X_train))
    sample_image = X_train[index].squeeze()
    plt.imshow(sample_image)
    
    sample_image = sample_image.reshape(-1, 32, 32, 1)
    result = sess.run(tf.argmax(logits, 1), feed_dict = { X : sample_image})
    result=result[0]
    print(get_name_from_label(result))

In [ ]:
with tf.Session() as sess:

    if __name__ == '__main__' :
        save_variables()
        restore_variables()